# The Solow Model with Alternative Specifications for Human Capital

This project explores the Solow Model with alternative specifications for Human Capital. First, I present the Solow Model with Human Capital, then I derive analytical and numerical steady-state solutions, visualize the outcomes, and analyze parameter dependence. Lastly, I investigate an alternative human capital specification involving two distinct worker groups: highly educated and unskilled.

Imports and set magics:

In [233]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# my modules
import random

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The Solow Model with Human Capital

The Solow Model with Human Capital is given by:
\begin{aligned}
    Y_{t} &= K^{\alpha}_{t}H^{\varphi}_{t}(A_{t}L_{t})^{1-\alpha-\varphi}, \quad 0<\alpha,\varphi<1 \hspace{4em} &(1)\\
    L_{t+1} &= (1+n)L_{t}, \quad L_{0} \text{ given} \hspace{4em} &(2)\\
    A_{t+1} &= (1+g)A_{t}, \quad A_{0}  \text{ given} \hspace{4em} &(3)\\
    K_{t+1} &= s_{K}Y_{t} + (1-\delta)K_{t}, \quad K_{0} \text{ given} \hspace{4em} &(4)\\
    H_{t+1} &= s_{H}Y_{t} + (1-\delta)H_{t}, \quad H_{0} \text{ given} \hspace{4em} &(5)
\end{aligned}

1. The Cobb-Douglas production function represents output, $Y_{t}$, as a function of physical capital, $K_{t}$, human capital, $H_{t}$, technology, $A_{t}$, and labor $L_{t}$. <br>
2. The labor force, $L_{t+1}$, accumulates over time according to population growth, $1+n$. <br>
3. Total factor productivity (TFP, represented as $A_{t}$), accumulates over time according to TFP growth, $1+g$. <br>
4. Physical capital, $K_{t+1}$, accumulates over time through investment in physical capital, $s_{K}Y_{t}$, and depreciation, ($1-\delta$). <br>
5. Human capital, $H_{t+1}$, accumulates over time through investment in human capital, $s_{H}Y_{t}$, and depreciation, $1-\delta$.

### Steady State Solutions

#### Analytical

The Solow Model with Human Capital has two Solow equations. The model is analyzed in tilde-variables, $\tilde{x_{t}} = \frac{X_{t}}{A_t{t}L_{t}}$, which are constant in steady state. The non-tilde per capita variables, $x_{t}$, change in steady state. 

The Solow equations are given by:

\begin{aligned}
    \tilde{k}_{t+1}-\tilde{k}_{t} &= \frac{1}{(1+n)(1+g)}(s_{K}\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}-(n+g+\delta+ng)\tilde{k}_{t}) \hspace{4em} &(6)\\
    \tilde{h}_{t+1}-\tilde{h}_{t} &= \frac{1}{(1+n)(1+g)}(s_{H}\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}-(n+g+\delta+ng)\tilde{h}_{t}) \hspace{4em} &(7)
\end{aligned}

In steady state $\tilde{k}_{t+1}=\tilde{k}_{t}=\tilde{k}^{\star}$ and $\tilde{h}_{t+1}=\tilde{h}_{t}=\tilde{h}^{\star}$. 






To solve for the steady state, I define the parameters as symbols, formulate steady-state Solow equations, and use `SymPy.solve`.

In [234]:
# Creates a SymPy symbol for each variable and parameter
k, h, s_K, s_H, n, g, delta, alpha, varphi = sm.symbols('k, h, s_K, s_H, n, g, delta, alpha, varphi')

# Sets the Solow equations in steady state as SumPy equations
ss_solow_k = sm.Eq(0, (1 / ((1 + n) * (1 + g))) * (s_K * k**alpha * h**varphi - (n + g + delta + n * g) * k))
ss_solow_h = sm.Eq(0, (1 / ((1 + n) * (1 + g))) * (s_H * k**alpha * h**varphi - (n + g + delta + n * g) * h))

# Try to solve the systems of equations algebraically for k and h using SumPy
try:
    ss_k_h = sm.solve([ss_solow_k, ss_solow_h], k, h)
except (NotImplementedError):
    print('Unable to solve the system of equations algebraically.')

Unable to solve the system of equations algebraically.


Due to the limitations of `SymPy` in solving the given system of non-linear equations algebraically, I employ an alternative approach. <br>
I find the solutions by utilizing the steady-state equations for capital and human capital, defined in "Introducing Advanced Macroeconomics" by Sørensen and Whitta-Jacobsen. <br>
I then create lambdified expressions for the equations and compute the steady-state values of capital and human capital.

The steady-state equations are given by:
\begin{aligned}
    \tilde{k}_{t}^{\star} = \left(\frac{s_K^{1-\varphi}s_H^{\varphi}}{n+g+\delta+ng}\right)^{\frac{1}{1-\alpha-\varphi}} \hspace{4em} (8) \\
    \tilde{h}_{t}^{\star} = \left(\frac{s_K^{\alpha}s_H^{1-\alpha}}{n+g+\delta+ng}\right)^{\frac{1}{1-\alpha-\varphi}} \hspace{4em} (9)
\end{aligned}

In [235]:
# Set the steady state equations
ss_eq_k = ((s_K**(1-varphi) * s_H**varphi) / (n + g + delta + n * g))**(1 / (1 - alpha - varphi))
ss_eq_h = ((s_K**(alpha) * s_H**(1-alpha)) / (n + g + delta + n * g))**(1 / (1 - alpha - varphi))

# Create a function for the steady state of k and h using SymPy lambdify
ss_func = sm.lambdify(args = (s_K, s_H, n, g, delta, alpha, varphi), expr = (ss_eq_k, ss_eq_h))

I set the name of the parameter values to the name of each parameter plus `_val` to not overwrite the original values.

In [236]:
# Set parameter values
s_K_val = 0.3
s_H_val = 0.2
n_val = 0.02
g_val = 0.02
delta_val = 0.1
alpha_val = 1/3
varphi_val = 1/3

Given the parameter values, I find the steady state values of capital and human capital.

In [237]:
# Call each steady state function for the parameter values
ss_k_func, ss_h_func = ss_func(s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val)

# Prints the steady state value for capital and human capital
print(f'Analytical solution:\n\
There are {ss_k_func:.3f} units of capital, and {ss_h_func:.3f} units of human capital in steady state.')

Analytical solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state.


#### Numerical

Now, that I have an analytical solution independent of any algorithm's initial values or tolerance, I can employ an optimizer to determine the steady state values for capital and human capital and compare the results to the analytical solution. 

In [238]:
# Call function to find the steady state of capital and human capital
from modelproject import n_ss

# Guess on values
initial_guess = [2, 2]

# Solve the function for steady state
sol = optimize.root(fun = n_ss, x0 = initial_guess, args = (s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val), method = 'hybr', tol = 0.0001)

# Save the values
num_ss_k, num_ss_h = sol.x

# Print the steady-state values for capital and human capital
print(f'Numerical Solution:\n\
There are {num_ss_k:.3f} units of capital, and {num_ss_h:.3f} units of human capital in steady state')

Numerical Solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state


Given a "good" initial guess, the numerical solutions are the same as the analytical ones. The downside to the simple optimizer is that it is highly sensitive to the initial guess, i.e., `initial_guess = [0.1, 1]` does not converge. To address this issue, I employ a multi-start optimization technique.
The multi-start optimization takes a specified number of random initial guesses within a bounded range for the variables. For each guess, it solves for the steady-state variables, calculates the residual of the function, and stores both the solution and the residual. Finally, it returns the solution with the smallest residual, corresponding to the best convergence or minimum error among all the initial guesses.
This approach improves the reliability of the optimization results by exploring multiple starting points and selecting the solution that provides the best convergence.

In [239]:
# Call the Multi-Start function to find the steady state of capital and human capital
from modelproject import multi_start

ms_ss_k, ms_ss_h, smallest_residual = multi_start(num_guesses=100,
            bounds=[0.1, 10],
            fun=n_ss, 
            args= (s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val), 
            method='hybr')

# Print the steady-state values for capital and human capital
print(f'Muti-Start Solution:\n\
There are {ms_ss_k:.3f} units of capital, and {ms_ss_h:.3f} units of human capital in steady state\n\
The functions residual is {smallest_residual}')

Muti-Start Solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state
The functions residual is 1.0671117114813116e-16


The solution shows that the Multi-Start optimizer is a highly stable method, consistently finding the correct answer. One can increase the number of guesses within the given bounds to increase precision. However, it is important to note that increasing the number of guesses can be time-consuming.

### Phase Diagrams

In [2]:
# Functions to determine null-clines 
def null_cline_of_k(k, h, s_K, s_H, n, g, delta, alpha, varphi):
    null_cline_k = (1 / ((1 + n) * (1 + g))) * (s_K * k**alpha * h**varphi - (n + g + delta + n * g) * k)
    return null_cline_k

def null_cline_of_h(k, h, s_K, s_H, n, g, delta, alpha, varphi):
    null_cline_h = (1 / ((1 + n) * (1 + g))) * (s_H * k**alpha * h**varphi - (n + g + delta + n * g) * h)
    return null_cline_h

# Create a Lambda function for each null-cline
func_null_cline_k = lambda h: null_cline_of_k(k, h, s_K, s_H, n, g, delta, alpha, varphi)

func_null_cline_h = lambda h: null_cline_of_h(k, h, s_K, s_H, n, g, delta, alpha, varphi) 


# Find roots

sol_k = optmize.root_scalar 



